In [2]:
oka = "coded by Ok@Nikolov"
okah = "coded by Ok@Nikolov with external help"

import pandas as pd

class Preisschaetzer:
    def __init__(self, excel_datei='6199.xlsx'):
        self.daten = pd.read_excel(excel_datei)

    def berechne_preis(self, eingaben):
        grundstück_faktor = self.daten['Faktor_Grundstück'].values[0]
        wohnfläche_faktor = self.daten['Faktor_Wohnfläche'].values[0]

        preis = (
            grundstück_faktor * eingaben['grundstück'] + 
            wohnfläche_faktor * eingaben['wohnfläche']
        )

        if eingaben['architektenhaus']:
            preis *= self.daten['Faktor_Architektenhaus'].values[0]
        if eingaben['makler']:
            preis *= self.daten['Faktor_Makler'].values[0]
        if eingaben['denkmalschutz']:
            preis *= self.daten['Faktor_Denkmalschutz'].values[0]

        if eingaben['lage'] == 'Stadt':
            preis *= self.daten['Faktor_Stadt'].values[0]
        else:
            preis *= self.daten['Faktor_Land'].values[0]

        preis *= self.daten['Faktor_Ausstattung'][eingaben['ausstattung']]
        preis *= self.daten['Faktor_Baujahr'][eingaben['baujahr']]
        preis *= self.daten['Faktor_Bundesland'][eingaben['bundesland']]
        preis *= self.daten['Faktor_Hausart'][eingaben['hausart']]

        return preis
    
oka

'coded by Ok@Nikolov'

**Erklärung des Backends für die Praktikanten Eric, Artjoms und Michael**

- Import der **Pandas-Bibliothek:** Pandas wird verwendet, um Excel-Dateien zu lesen und zu verarbeiten, da es eine leistungsfähige Datenstruktur namens DataFrame bereitstellt.

- Klassenname **Preisschaetzer:** Dies ist eine Klasse, die eine Excel-Datei mit Preisfaktoren lädt und basierend auf Benutzereingaben den Immobilienpreis berechnet.
- *Initialisierung der Klasse:*
- Die Methode __init__ wird aufgerufen, wenn eine Instanz der Klasse erstellt wird.
- Parameter excel_datei: Der Standardwert ist die Datei 6199.xlsx, die in derselben Verzeichnisstruktur wie das Python-Skript gespeichert werden sollte.
- self.daten: Hier wird die Excel-Datei gelesen und die Daten in einem DataFrame gespeichert, auf den später zugegriffen wird. Pandas wird verwendet, um die Datei zu lesen (pd.read_excel(excel_datei)).
- **berechne_preis:** Diese *Methode* berechnet den Preis der Immobilie basierend auf:
- eingaben: Ein *Dictionary*, das die Eingabewerte des Benutzers enthält (z.B. Grundstücksgröße, Wohnfläche, besondere Faktoren wie Denkmalschutz).
- **Lagefaktor:** Der Preis wird basierend auf der Lage der Immobilie (Stadt oder Land) angepasst.
- *Faktor_Stadt:* Dieser Wert aus der Excel-Datei wird verwendet, wenn die Immobilie in der Stadt liegt.
- *Faktor_Land:* Dieser Wert wird verwendet, wenn die Immobilie auf dem Land liegt.
- Ausstattung: Der Preis wird mit einem Faktor multipliziert, der von der Ausstattung der Immobilie abhängt.
- *Baujahr:* Der Faktor für das Baujahr wird berücksichtigt.
- *Bundesland:* Der Faktor variiert je nach Bundesland.
- *Hausart:* Verschiedene Hausarten haben unterschiedliche Faktoren.
- *Preis zurückgeben:* Die Methode gibt den endgültig berechneten Immobilienpreis zurück.




**Quellen:**
Das Backend wurde mithilfe folgender Quellen erstellt
1. Eric's Unterricht
2. lerneprogrammieren.de (Letzter Zugriff 17.09.2024)
3. adesso.de (Letzter Zugriff 17.09.2024)
4. t2informatik.de (Letzter Zugriff 17.09.2024)
5. openai.com/chatgtp (Letzter Zugriff 17.09.2024)
6. Backend (FULL) Kurs via Youtube: https://www.youtube.com/watch?v=YyZzX2nWTwA (Letzter Zugriff 14.09.2024)

-

**Allerdings greift der Code, den wir oben gesehen haben auf fest codierte Spaltennamen zu**  Damit der Code *flexibel* bleibt und sich an Änderungen der Werte in der Excel-Datei anpasst, werde ich ihn so  gestalten, dass die Spaltennamen und die Werte *dynamisch aus der Excel-Datei* ausgelesen werden. Dies bedeutet, dass wir nicht fest codierte Spaltennamen verwenden, sondern die *Excel-Datei dynamisch* analysieren, um die passenden Werte zu extrahieren.

In [7]:
import pandas as pd

# Hier seht ihr die generelle Backend-Logik: Der Code liest Daten aus der Excel-Datei und berechnet den Preis
class PreisSchätzer:
    def __init__(self, excel_datei):
        self.daten = pd.read_excel(excel_datei, sheet_name=None)  
        # Liest alle Blätter der Excel-Datei - beim Zusammenfügen von Frontent und Backend müssen wir das noch auf das eine Tabellenblatt/ Zellen anpassen (in unserem Fall = Tabelle 1 - die unteren Strings sind also später hinfällig- der Vollständigkeit werden sie mit aufgenommen)
        self.parameter = self.daten['Parameter']  # Platzhalter1: Tabelle 'Parameter' enthält relevante Faktoren
        self.faktoren = self.daten['Faktoren']    # Platzhalter2: Tabelle 'Faktoren' enthält Berechnungsfaktoren

    
    # Dynamisches Abrufen von Faktoren aus der Excel-Datei der Faktoren, die uns vorgegeben wurden. Ich werde nicht jeden einzelnen String benennen, bei Fragen kommt auf mich zu 
    # Die Variabeln sollten Eindeutig benannt sein 
    def berechne_preis(self, eingaben): 
       
        grundstück_faktor = self._hole_faktor('Grundstück', eingaben['grundstück'])
        wohnfläche_faktor = self._hole_faktor('Wohnfläche', eingaben['wohnfläche'])

        preis = grundstück_faktor * eingaben['grundstück'] + wohnfläche_faktor * eingaben['wohnfläche']

        if eingaben['architektenhaus']:
            preis *= self._hole_faktor('Architektenhaus', True)
        if eingaben['makler']:
            preis *= self._hole_faktor('Makler', True)
        if eingaben['denkmalschutz']:
            preis *= self._hole_faktor('Denkmalschutz', True)

        lage_faktor = self._hole_faktor('Lage', eingaben['lage'])
        preis *= lage_faktor

        ausstattungs_faktor = self._hole_faktor('Ausstattung', eingaben['ausstattung'])
        baujahr_faktor = self._hole_faktor('Baujahr', eingaben['baujahr'])
        bundesland_faktor = self._hole_faktor('Bundesland', eingaben['bundesland'])
        hausart_faktor = self._hole_faktor('Hausart', eingaben['hausart'])

        preis *= ausstattungs_faktor * baujahr_faktor * bundesland_faktor * hausart_faktor

        return preis

    def _hole_faktor(self, kategorie, wert):
        try:
            faktor = self.faktoren.loc[self.faktoren['Kategorie'] == kategorie, str(wert)].values[0]
        except IndexError:
            raise ValueError(f"Kein Faktor für {kategorie} und Wert {wert} gefunden.")
        return faktor
    
okah


'coded by Ok@Nikolov with external help'

**Wichtige Änderungen:**
1.	*Dynamische Struktur:* Anstatt fest definierte Spaltennamen zu verwenden (z.B. Faktor_Grundstück, Faktor_Wohnfläche), wird die Excel-Datei analysiert, um die relevanten Faktoren dynamisch basierend auf den Kategorien (z.B. Grundstück, Wohnfläche) und den Werten des Nutzers zu finden.
2.	*Flexibilität bei der Excel-Struktur:* Die Excel-Datei kann erweitert oder verändert werden, solange die Blattnamen (z.B. Parameter und Faktoren) beibehalten werden. Dies ermöglicht es, dass der Code weiterhin funktioniert, auch wenn sich die Werte oder die Kategorien ändern.
3.	*_hole_faktor Methode:* Diese Methode sucht dynamisch nach dem entsprechenden Faktor für eine bestimmte Kategorie (wie Grundstück) und einen bestimmten Wert (wie die Grundstücksgröße). Wenn kein passender Faktor gefunden wird, wirft die Methode einen Fehler, damit klar ist, dass etwas mit der Eingabe nicht stimmt.


*Die Codeänderungen wurde mithilfe folgender Quellen erstellt:*
1. Eric's Unterricht
5. openai.com/chatgtp [ hat mir vorallem bei der hole_faktor Methode geholfen] (Letzter Zugriff 17.09.2024)
6. Backend (FULL) Kurs via Youtube: https://www.youtube.com/watch?v=YyZzX2nWTwA (Letzter Zugriff 14.09.2024)